In [1]:
! pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 32.6 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import root
from rdkit import Chem
from rdkit.Chem import Descriptors

# =================================================================
# 1. PARÁMETROS DE ENTRADA (DISEÑO DE PROCESO)
# =================================================================
MASA_ACEITE_IN = 100.0   # kg (Aceite de Palma - Trioleína)
RATIO_MOLAR = 6.0        # Moles Metanol / Mol Aceite
CATALIZADOR_PCT = 0.8    # % en peso de KOH respecto al aceite
TEMP_REACCION_C = 60.0   # Grados Celsius
CONVERSION_TARGET = 0.98 # 98% de conversión

# SMILES para procesamiento molecular
SMILES = {
    'Aceite': 'CCCCCCCC=CCCCCCCCC(=O)OCC(OC(=O)CCCCCCCC=CCCCCCCCC)COC(=O)CCCCCCCC=CCCCCCCCC',
    'Metanol': 'CO',
    'FAME': 'CCCCCCCC=CCCCCCCCC(=O)OC',
    'Glicerol': 'OCC(O)CO'
}

class SimuladorProcesos:
    def __init__(self):
        self.mw = {k: Descriptors.MolWt(Chem.MolFromSmiles(v)) for k, v in SMILES.items()}

    def fragmentar(self):
        """Fragmentación automática para reporte técnico"""
        mol = Chem.MolFromSmiles(SMILES['Aceite'])
        return {
            'CH3': len(mol.GetSubstructMatches(Chem.MolFromSmarts("[CH3]"))),
            'CH2': len(mol.GetSubstructMatches(Chem.MolFromSmarts("[CH2]"))),
            'C=C': len(mol.GetSubstructMatches(Chem.MolFromSmarts("[CH]=[CH]"))),
            'COO': len(mol.GetSubstructMatches(Chem.MolFromSmarts("C(=O)O")))
        }

    def calcular_nrtl_gamma(self, x, T):
        """Cálculo manual de coeficientes de actividad NRTL"""
        nc = 3 # FAME, Glicerol, Metanol
        # Parámetros Aij (cal/mol) - Datos de literatura para este sistema
        Aij = np.array([[0.0, 1800.0, 600.0], [2500.0, 0.0, 450.0], [500.0, -100.0, 0.0]])
        alpha = 0.2 * np.ones((nc, nc))
        np.fill_diagonal(alpha, 0)

        tau = Aij / (1.987 * T) # Usando constante R en cal/molK
        G = np.exp(-alpha * tau)

        gamma = np.zeros(nc)
        for i in range(nc):
            sum1 = np.sum(x * tau[:, i] * G[:, i])
            sum2 = np.sum(x * G[:, i])
            term2 = 0
            for j in range(nc):
                den_j = np.sum(x * G[:, j])
                if den_j > 0:
                    inner = tau[i, j] - (np.sum(x * tau[:, j] * G[:, j]) / den_j)
                    term2 += (x[j] * G[i, j] / den_j) * inner
            gamma[i] = np.exp(sum1/sum2 + term2)
        return gamma

    def ejecutar(self):
        # --- A. BALANCE DE MATERIA ---
        n_aceite = (MASA_ACEITE_IN * 1000) / self.mw['Aceite']
        n_meoh_feed = n_aceite * RATIO_MOLAR
        m_meoh_in = (n_meoh_feed * self.mw['Metanol']) / 1000
        m_koh = MASA_ACEITE_IN * (CATALIZADOR_PCT / 100)

        n_reacc = n_aceite * CONVERSION_TARGET
        m_fame = (n_reacc * 3 * self.mw['FAME']) / 1000
        m_gly = (n_reacc * self.mw['Glicerol']) / 1000
        m_meoh_res = ((n_meoh_feed - 3*n_reacc) * self.mw['Metanol']) / 1000
        m_aceite_res = ((n_aceite - n_reacc) * self.mw['Aceite']) / 1000

        # --- B. BALANCE ENERGÉTICO ---
        # Cp promedio (kJ/kgK)
        cp_aceite, cp_meoh = 2.1, 2.53
        q_calentamiento = (MASA_ACEITE_IN * cp_aceite + m_meoh_in * cp_meoh) * (TEMP_REACCION_C - 20)
        delta_h_rxn = -15.5 * n_reacc / 1000 # MJ (Exotérmica)

        # --- C. SEPARACIÓN DE FASES (NRTL) ---
        # Composición molar global para el flash
        n_total = (m_fame/self.mw['FAME'] + m_gly/self.mw['Glicerol'] + m_meoh_res/self.mw['Metanol']) * 1000
        z = [ (m_fame*1000/self.mw['FAME'])/n_total, (m_gly*1000/self.mw['Glicerol'])/n_total, (m_meoh_res*1000/self.mw['Metanol'])/n_total ]

        # --- IMPRESIÓN DEL REPORTE TÉCNICO ---
        print("="*70)
        print("   REPORTE TÉCNICO DE SIMULACIÓN: PLANTA PILOTO BIODIÉSEL PALMA")
        print("="*70)

        print(f"\n[1] CARACTERIZACIÓN DE MATERIA PRIMA (SMILES)")
        print(f"Especie: Trioleína (C57H104O6)")
        frag = self.fragmentar()
        for k, v in frag.items():
            print(f" - Grupo {k:<5}: {v} unidades")

        print(f"\n[2] BALANCE DE MATERIA (Base: {MASA_ACEITE_IN} kg Aceite)")
        print(f"{'-'*65}")
        print(f"{'Componente':<20} | {'Entrada (kg)':<15} | {'Salida (kg)':<15}")
        print(f"{'-'*65}")
        print(f"{'Aceite Palma':<20} | {MASA_ACEITE_IN:<15.3f} | {m_aceite_res:<15.3f}")
        print(f"{'Metanol':<20} | {m_meoh_in:<15.3f} | {m_meoh_res:<15.3f}")
        print(f"{'Catalizador KOH':<20} | {m_koh:<15.3f} | {m_koh:<15.3f}")
        print(f"{'Biodiésel (FAME)':<20} | {'0.000':<15} | {m_fame:<15.3f}")
        print(f"{'Glicerol':<20} | {'0.000':<15} | {m_gly:<15.3f}")
        print(f"{'-'*65}")
        m_in_tot = MASA_ACEITE_IN + m_meoh_in + m_koh
        m_out_tot = m_fame + m_gly + m_meoh_res + m_aceite_res + m_koh
        print(f"{'TOTAL TOTAL':<20} | {m_in_tot:<15.3f} | {m_out_tot:<15.3f}")

        print(f"\n[3] INDICADORES DE RENDIMIENTO (KPIs)")
        print(f" - Rendimiento en masa (kg FAME / kg Aceite): {m_fame/MASA_ACEITE_IN:.3f}")
        print(f" - Conversión Molar: {CONVERSION_TARGET*100:.1f} %")
        print(f" - Producción de Glicerina: {m_gly:.2f} kg")

        print(f"\n[4] BALANCE ENERGÉTICO")
        print(f" - Calor sensible para calentamiento (20->60°C): {q_calentamiento/1000:.2f} MJ")
        print(f" - Calor de reacción (Exotérmico): {abs(delta_h_rxn):.2f} MJ")
        print(f" - Carga térmica neta del reactor: {(q_calentamiento/1000) + delta_h_rxn:.2f} MJ")

        print(f"\n[5] TERMODINÁMICA DE SEPARACIÓN (NRTL)")
        gamma = self.calcular_nrtl_gamma(np.array(z), TEMP_REACCION_C + 273.15)
        print(f"Coeficientes de Actividad (Gamma) en la mezcla:")
        print(f" - Gamma FAME:     {gamma[0]:.4f}")
        print(f" - Gamma Glicerol: {gamma[1]:.4f} (Alta no-idealidad)")
        print(f" - Gamma Metanol:  {gamma[2]:.4f}")

# Ejecutar el simulador
sim = SimuladorProcesos()
sim.ejecutar()


   REPORTE TÉCNICO DE SIMULACIÓN: PLANTA PILOTO BIODIÉSEL PALMA

[1] CARACTERIZACIÓN DE MATERIA PRIMA (SMILES)
Especie: Trioleína (C57H104O6)
 - Grupo CH3  : 3 unidades
 - Grupo CH2  : 43 unidades
 - Grupo C=C  : 3 unidades
 - Grupo COO  : 3 unidades

[2] BALANCE DE MATERIA (Base: 100.0 kg Aceite)
-----------------------------------------------------------------
Componente           | Entrada (kg)    | Salida (kg)    
-----------------------------------------------------------------
Aceite Palma         | 100.000         | 2.000          
Metanol              | 22.062          | 11.252         
Catalizador KOH      | 0.800           | 0.800          
Biodiésel (FAME)     | 0.000           | 95.299         
Glicerol             | 0.000           | 10.357         
-----------------------------------------------------------------
TOTAL TOTAL          | 122.862         | 119.707        

[3] INDICADORES DE RENDIMIENTO (KPIs)
 - Rendimiento en masa (kg FAME / kg Aceite): 0.953
 - Conversión